In [ ]:
!pip install -q tensorflow-gpu==2.0.0-rc1

In [1]:
import os
from datetime import datetime
import tensorflow as tf
import numpy as np
import json
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
base_path = '/content/gdrive/My Drive/Colab Notebooks'

In [6]:
data_in_path = base_path + '/data_in/'
data_out_path = base_path + '/data_out/'
train_input_data = 'nsmc_train_input.npy'
train_label_data = 'nsmc_train_label.npy'
data_configs = 'nsmc_data_configs.json'

input_data = np.load(open(data_in_path + train_input_data, 'rb'))
label_data = np.load(open(data_in_path + train_label_data, 'rb'))
prepro_configs = json.load(open(data_in_path + data_configs, 'r'))

In [7]:
test_split = 0.1
seed = 100
batch_size = 50
num_epochs = 16

input_train, input_eval, label_train, label_eval = train_test_split(input_data,
                                                                    label_data,
                                                                    test_size=test_split,
                                                                    random_state=seed)

In [23]:
model_name = 'convnet'
max_length = input_data.shape[1]

In [8]:
dataset = tf.data.Dataset.from_tensor_slices((input_train, label_train))
dataset = dataset.shuffle(buffer_size=len(input_train))
dataset = dataset.batch(batch_size)
dataset = dataset.shuffle(len(input_train))

validation_dataset = tf.data.Dataset.from_tensor_slices((input_eval, label_eval))
validation_dataset = validation_dataset.shuffle(buffer_size=len(input_eval))
validation_dataset = validation_dataset.batch(batch_size)
dataset = dataset.shuffle(len(input_eval))

In [9]:
from tensorflow.keras import layers

In [10]:
class Model(tf.keras.Model):
    
    def __init__(self, **kargs):
        super(Model, self).__init__(name='conv')
        self.embedding = layers.Embedding(input_dim=kargs['vocab_size'],
                                     output_dim=kargs['embedding_size'])
        self.conv_list = [layers.Conv1D(filters=kargs['num_filters'],
                                   kernel_size=kernel_size,
                                   padding='valid',
                                   activation=tf.keras.activations.relu,
                                   kernel_constraint=tf.keras.constraints.MaxNorm(max_value=3.))
                     for kernel_size in [3,4,5]]
        self.pooling = layers.GlobalMaxPooling1D()
        self.dropout = layers.Dropout(kargs['dropout_rate'])
        self.fc1 = layers.Dense(units=kargs['hidden_dimension'],
                           activation=tf.keras.activations.relu,
                           kernel_constraint=tf.keras.constraints.MaxNorm(max_value=3.))
        self.fc2 = layers.Dense(units=kargs['output_dimension'],
                           activation=tf.keras.activations.sigmoid,
                           kernel_constraint=tf.keras.constraints.MaxNorm(max_value=3.))
    
    def call(self, x):
        x = self.embedding(x)
        x = self.dropout(x)
        x = tf.concat([self.pooling(conv(x)) for conv in self.conv_list], axis=-1)
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x

In [11]:
kargs = {'vocab_size': prepro_configs['vocab_size'],
        'embedding_size': 128,
        'num_filters': 100,
        'dropout_rate': 0.5,
        'hidden_dimension': 250,
        'output_dimension':1}

In [13]:
model = Model(**kargs)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [25]:
model.build(input_shape=(batch_size, max_length))
model.summary()

Model: "conv"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  5600896   
_________________________________________________________________
conv1d_3 (Conv1D)            multiple                  38500     
_________________________________________________________________
conv1d_4 (Conv1D)            multiple                  51300     
_________________________________________________________________
conv1d_5 (Conv1D)            multiple                  64100     
_________________________________________________________________
global_max_pooling1d_1 (Glob multiple                  0         
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
_________________________________________________________________
dense_2 (Dense)              multiple                  75250  

In [21]:
class Model(tf.keras.Model):
    
    def __init__(self, **kargs):
        super(Model, self).__init__(name='conv')
        self.embedding = layers.Embedding(input_dim=kargs['vocab_size'],
                                     output_dim=kargs['embedding_size'])
        self.conv_list = [layers.Conv1D(filters=kargs['num_filters'],
                                   kernel_size=kernel_size,
                                   padding='valid',
                                   activation=tf.keras.activations.relu,
                                   kernel_constraint=tf.keras.constraints.MaxNorm(max_value=3.))
                     for kernel_size in [3,4,5]]
        self.pooling = layers.GlobalMaxPooling1D()
        self.dropout = layers.Dropout(kargs['dropout_rate'])
        self.fc1 = layers.Dense(units=kargs['hidden_dimension'],
                           activation=tf.keras.activations.relu,
                           kernel_constraint=tf.keras.constraints.MaxNorm(max_value=3.))
        self.fc2 = layers.Dense(units=kargs['output_dimension'],
                           activation=tf.keras.activations.sigmoid,
                           kernel_constraint=tf.keras.constraints.MaxNorm(max_value=3.))
    
    def call(self, x):
        x = self.embedding(x)
        x = self.dropout(x)
        x = tf.concat([self.pooling(conv(x)) for conv in self.conv_list], axis=-1)
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x
    
    def build_graph(self, input_shape):
        input_shape_wo_batch = input_shape[1:]
        self.build(input_shape)
        inputs = tf.keras.Input(shape=input_shape_wo_batch)
        
        _ = self.call(inputs)

In [26]:
model = Model(**kargs)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [28]:
model.build_graph(input_shape=(batch_size, max_length))
model.summary()

Model: "conv"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 128)           5600896   
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 48, 100)           38500     
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 47, 100)           51300     
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 46, 100)           64100     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 100)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 50, 128)           0         
_________________________________________________________________
dense_4 (Dense)              (None, 250)               75250  

In [30]:
checkpoint_path = data_out_path + model_name + '/weights.{epoch:02d}-{val_loss:.2f}'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True)

model.fit(dataset, epochs=num_epochs,
         validation_data=validation_dataset,
         callbacks=[cp_callback])

Epoch 1/16
    208/Unknown - 12s 57ms/step - loss: 0.1932 - accuracy: 0.9235

KeyboardInterrupt: 

In [49]:
test_input_data = 'nsmc_test_input.npy'
test_label_data = 'nsmc_test_label.npy'

test_input_data = np.load(open(data_in_path + test_input_data, 'rb'))
test_label_data = np.load(open(data_in_path + test_label_data, 'rb'))

In [51]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_input_data, test_label_data))
test_dataset = test_dataset.batch(BATCH_SIZE)

In [53]:
test_output = model.predict(test_dataset)

In [54]:
test_output = np.array(test_output)

In [55]:
test_output

array([[0.9836465 ],
       [0.54142475],
       [0.26841763],
       ...,
       [0.7737855 ],
       [0.00120722],
       [0.0788133 ]], dtype=float32)